In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
env = gym.make('CartPole-v0')
observation = env.reset()

In [20]:
parameters = np.random.rand(4) * 2 - 1 

def run_episode(env, parameters):  
    observation = env.reset()
    totalreward = 0
    while True:

        action = 0 if np.matmul(parameters, observation) < 0 else 1
        observation, reward, done, info = env.step(action)
        totalreward += reward
        if done:
            break
    return totalreward

run_episode(env, parameters)

23.0

### Random search

In [27]:
bestparams = None  
bestreward = 0  


for _ in range(10000):  
    parameters = np.random.rand(4) * 2 - 1
    reward = run_episode(env, parameters)
    if reward > bestreward:
        bestreward = reward
        bestparams = parameters
        # considered solved if the agent lasts 200 timesteps
        if reward == 200:
            break

### Hill climbing algorithm

In [64]:
noise_scaling = 0.1  
parameters = np.random.rand(4) * 2 - 1  
bestreward = 0  
for _ in range(10000):  
    newparams = parameters + (np.random.rand(4) * 2 - 1)*noise_scaling
    reward = 0
    run = run_episode(env,newparams)
    if reward > bestreward:
        bestreward = reward
        parameters = newparams
        if reward == 200:
            break

### Policy gradient

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Bernoulli
from torch.autograd import Variable
from itertools import count
import matplotlib.pyplot as plt
import numpy as np
import gym
import pdb


class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(4, 1)
     
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        return x


In [27]:
episode_durations = []
def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.FloatTensor(episode_durations)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

# Parameters
num_episode = 5000
batch_size = 5
learning_rate = 0.01
gamma = 0.99

env = gym.make('CartPole-v0')
policy_net = PolicyNet()
optimizer = torch.optim.Adam(policy_net.parameters(), lr=learning_rate)

# Batch History
state_pool = []
action_pool = []
reward_pool = []
steps = 0

In [46]:
state = env.reset()
state = torch.from_numpy(state).float()
state = Variable(state)
policy_net(state)

tensor([0.5119], grad_fn=<SigmoidBackward>)

In [104]:
probs = policy_net(state)
m = Bernoulli(probs)

action = m.sample()

action = action.data.numpy().astype(int)[0]
action

1

In [50]:
for e in range(num_episode):
    state = env.reset()
    state = torch.from_numpy(state).float()
    state = Variable(state)
    # env.render(mode='rgb_array')
    
    
    for t in count():

            probs = policy_net(state)
            m = Bernoulli(probs)
            action = m.sample()
        
            action = action.data.numpy().astype(int)[0]
            next_state, reward, done, _ = env.step(action)
            #env.render(mode='rgb_array')

            # To mark boundarys between episodes
            if done:
                reward = 0

            state_pool.append(state)
            action_pool.append(float(action))
            reward_pool.append(reward)

            state = next_state
            state = torch.from_numpy(state).float()
            state = Variable(state)

            steps += 1

            if done:
                episode_durations.append(t + 1)
                plot_durations()
                break

        # Update policy
        if e > 0 and e % batch_size == 0:

            # Discount reward
            running_add = 0
            for i in reversed(range(steps)):
                if reward_pool[i] == 0:
                    running_add = 0
                else:
                    running_add = running_add * gamma + reward_pool[i]
                    reward_pool[i] = running_add

            # Normalize reward
            reward_mean = np.mean(reward_pool)
            reward_std = np.std(reward_pool)
            for i in range(steps):
                reward_pool[i] = (reward_pool[i] - reward_mean) / reward_std

            # Gradient Desent
            optimizer.zero_grad()

            for i in range(steps):
                state = state_pool[i]
                action = Variable(torch.FloatTensor([action_pool[i]]))
                reward = reward_pool[i]

                probs = policy_net(state)
                m = Bernoulli(probs)
                loss = -m.log_prob(action) * reward  # Negtive score function x reward
                loss.backward()

            optimizer.step()

            state_pool = []
            action_pool = []
            reward_pool = []
            steps = 0


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 38)